# Removing Noise from low frequency OBS data

## Set up local paths and functions to read data/metadata either locally or online

In [ ]:
%load_ext autoreload
%autoreload 2

from read_data_metadata import read_data, read_metadata, DATA_PATH

## Import Data and Metadata

In [ ]:
from obspy.clients.fdsn import Client
from obspy import Stream

# Declare parameters
nslc, st, et, server = "7D.G03A.*.L*", "2012-02-05T00", "2012-02-13T00", "IRIS"  # Cascadia Expt: From Bell et al. [2015]
nslc, st, et, server = "7D.G02B.*.B*", "2013-02-05T00", "2013-02-13T00", "IRIS"  # Cascadia Expt
nslc, st, et, server = "Z3.A422A.*.B*", "2017-12-25T00", "2017-12-30T00", "RESIF"  # AlpArray Expt

# Read data and metadata
stream = read_data(server, nslc, st, et)
inv = read_metadata(server, nslc, st, et)

# Optional: print data to make sure it's ok
print(stream.__str__(extended=True))
print(inv)

# Trim data down to the main seismometer + pressure channels
channels = ['*HZ', '*H1', '*H2', '*DH']
stream_essential = Stream(traces=[])
for ch in channels:
    stream_essential.extend(stream.select(channel=ch))

# Optional: print data to make sure it's ok
print(stream_essential.__str__(extended=True))

# Write data (miniSEED) and metadata (StationXML) to local disk
stream_essential.write(DATA_PATH / 'example.mseed', 'MSEED')
inv.write(DATA_PATH / 'example.station.xml', 'STATIONXML')

## Downsampling the data

As we are only worried about data below 1 Hz, I downsample to make the processing faster.
But this changes the channel name and the instrument response, so I have to update the inventory.
The tiskitpy [Decimator class](https://tiskitpy.readthedocs.io/latest/classes/decimator.html)
has a method for this, plus a shell script for directly downsampling a SDS repository
and updating its associated inventory

In [ ]:
from obspy import read, read_inventory
from tiskitpy import Decimator

# Read in data and inventory
stream = read(DATA_PATH / "example.mseed", "MSEED")
inv = read_inventory(DATA_PATH / "example.station.xml", "STATIONXML")

# Decimate data and update inventory
decimator = Decimator((5, 5, 2))  # decimate by 50 overall
stream_decim = decimator.decimate(stream)
inv_decim = decimator.update_inventory(inv)

# Write out decimated data and updated inventory
stream_decim.write(DATA_PATH / 'example_decim.mseed', 'MSEED')
inv_decim.write(DATA_PATH / 'example_decim.station.xml', 'STATIONXML')
print('All Done')s

In [ ]:
print(stream)
print(stream_decim)

## Simple rotation

Rotating the vertical channel to true vertical does not deform the signal, it simply shifts the signal between channels.
We use the variance of the data to determine the best correction angle.

> *Note: if there is no tilt noise or if there is another signal much stronger than the noise, this method will give nonsense results.*

tiskitpy's ``CleanRotator`` class does this for you, but the method is so simple that I prefer us to do it ourselves.

### Create a function to calculate variance as a function of the tilt correction


In [ ]:
import numpy as np
from obspy.signal.rotate import rotate2zne

def rotZ(Z, N, E, angle, azimuth):
    """
    Rotate traces by the given angle, azimuth

    Arguments:
        Z (:class:`opsPy.stream.Trace): Z channel trace
        N (:class:`opsPy.stream.Trace): N (or 1) channel trace
        E (:class:`opsPy.stream.Trace): E (or 2) channel trace
        angle (float): angle of Z channel from the vertical, azimuth (in degrees)
        azimuth (float): azimuth at which Z channel is offset from the vertical
    """
    # Missing checks to make sure all channels are same length and sampfreq
    # Protect input data
    Z, N, E = Z.copy(),  N.copy(), E.copy()

    # Rotate data
    [Z.data, N.data, E.data] = rotate2zne(Z.data, azimuth, -90 + angle,
                                          N.data, 0, angle * np.cos(np.deg2rad(azimuth)),
                                          E.data, 90, angle * np.sin(np.deg2rad(azimuth)))
    return Z, N, E

def rotZ_variance(angles, Z, N, E):
    """
    Calculate the variance for a given rotation

    Arguments:
        angles (list): angle, azimuth (in degrees)
        Z (:class:`opsPy.stream.Trace): Z channel trace
        N (:class:`opsPy.stream.Trace): N (or 1) channel trace
        E (:class:`opsPy.stream.Trace): E (or 2) channel trace

    Data should already be filtered into relevant band
    """
    # Missing checks to make sure all channels are same length and sampfreq

    # Rotate data
    Z, N, E = rotZ(Z, N, E, angles[0], angles[1])
    # Calculate variance
    var = np.sum(Z.data**2)
    return var


### Find the best tilt correction angle

In [ ]:
from obspy import Stream, read, read_inventory
import scipy as sp

# Read in data and metadata
stream = read('data/example_decim.mseed')
inv = read_inventory('data/example_decim.station.xml')

# Filter data to remove microseisms
filt_band = (.001, 0.01)
filt_stream = stream.copy()
filt_stream.detrend("demean")
filt_stream.detrend("linear")
filt_stream.filter("lowpass", freq=filt_band[1], corners=4, zerophase=True)
filt_stream.filter("highpass", freq=filt_band[0], corners=4, zerophase=True)

# Calculate best angle
Z, N, E = filt_stream.select(component='Z')[0], filt_stream.select(component='1')[0], filt_stream.select(component='2')[0]
start_variance = rotZ_variance([0, 0], Z, N, E)
print(f"Starting variance = {start_variance:5.5g}")
xopt, fopt, iter, funcalls, warnflag, allvecs = sp.optimize.fmin(
    func=rotZ_variance, x0=[0, 0], args=(Z, N, E), disp=False, full_output=True, retall=True)
bestAngle, bestAzimuth = xopt
best_variance = rotZ_variance([bestAngle, bestAzimuth], Z, N, E)
print(f"{bestAngle=:.1f}, {bestAzimuth=:.1f}, {best_variance=:5.5g}, variance reduction={100.*((start_variance-best_variance)/start_variance):.1f}%")

### Plot the results

In [ ]:
from tiskitpy import SpectralDensity

# Compare original and "best-angle" data
Z, N, E = stream.select(component='Z')[0], stream.select(component='1')[0], stream.select(component='2')[0]
new_stream = Stream(traces=[x for x in rotZ(Z, N, E, bestAngle, bestAzimuth)])
for tr in new_stream:
    tr.stats.location="BA"
compare_stream = stream.select(channel='*HZ') + new_stream.select(channel='*HZ')
sd = SpectralDensity.from_stream(compare_stream)
sd.plot(overlay=True)

## Transfer function-based tilt and compliance removal

We use tiskitpy's ``DataCleaner`` class.  Other options include ATACR ([python](https://nfsi-canada.github.io/OBStools/atacr.html) and [Matlab](https://github.com/helenjanisz/ATaCR) versions)

Below is a simple example with the same data and compare the result with that obtained using simple rotation.
The ``SpectralDensity`` class allows us to compare the power spectral densities of the original and modified traces

## Transfer-function based noise removal

You can remove several types of noise by subtracting their calculated levels based on the noise observed on a coherent channel.

For OBSs, this has been used to remove tilt noise from the vertical channel (using the horizontal channels as the "coherent channel"
and to remove seafloor compliance "noise" from the vertical channel (using the pressure channel as the "coherent channel"

This is a "sledgehammer" approach and can remove useful signals if you are not careful!  For example, if the transfer function is applied in the microseism band, you will generally remove Rayleigh waves (ambient noise and EQ) from your signal!

### Removing tilt noise

Compared with simple rotation

In [ ]:
from tiskitpy import DataCleaner, CleanRotator, CleanedStream

# Clean the data in three ways: rotation, transfer function, and rotation + transfer function
cr = CleanRotator(stream, H_over_Z=0.225)
stream_rot = cr.apply(stream)
# stream_rot = CleanRotator(stream).apply(stream)
stream_trans = DataCleaner(stream, ['*1', '*2'], max_freq=0.1).apply(stream)
stream_rot_trans = DataCleaner(stream_rot, ['*1', '*2'], max_freq=0.1).apply(stream_rot)

all_Zs = CleanedStream(traces=[x.select(component='Z')[0] for x in (stream, stream_rot, stream_trans, stream_rot_trans)])
sd_all = SpectralDensity.from_stream(all_Zs, inv=inv)
sd_all.plot(overlay=True)

### Removing seafloor compliance

Seafloor compliance is the quasi-static motion of the seafloor beneath ocean surface gravity waves.  In the deep ocean, only ocean surface
waves with periods > ~20s generate a detectable seafloor pressure, and compliance is limited to periods longer than this.

Compliance depends on the subsurface structure (down to up to 8 km beneath the seafloor), especially the shear modulus.
You can use compliance to estimate the subsurface shear modulus/velocity structure or, if you want to study something else,
you may want to remove it from your signal.  

Simple rotation does not removes compliance noise, but it can help the transfer function method to work.
Here's an example 

In [ ]:
from tiskitpy import DataCleaner, CleanRotator, SpectralDensity

# Clean the data in three ways: rotation, transfer function, and rotation + transfer function
stream_rot = CleanRotator(stream, H_over_Z=0.225).apply(stream)
stream_trans_H = DataCleaner(stream, ['*1', '*2'], max_freq=0.1).apply(stream)  # Remove horizontal channels
stream_trans_HP = DataCleaner(stream, ['*1', '*2', '*H'], max_freq=0.1).apply(stream)  # Remove horizontal channels
stream_rot_trans_H = DataCleaner(stream_rot, ['*1', '*2'], max_freq=0.1).apply(stream_rot)  # Rotate then remove horizontal channels
stream_rot_trans_HP = DataCleaner(stream_rot, ['*1', '*2', '*H'], max_freq=0.1).apply(stream_rot)  # Rotate then remove horiz & pressure
stream_rot_trans_P = DataCleaner(stream_rot, ['*H'], max_freq=0.1).apply(stream_rot)  # Rotate then remove horiz & pressure

all_Zs = Stream(traces=[x.select(component='Z')[0] for x in (stream, stream_rot, stream_trans_H, stream_trans_HP, stream_rot_trans_H, stream_rot_trans_HP, stream_rot_trans_P)])
sd_all = SpectralDensity.from_stream(all_Zs, inv=inv)
sd_all.plot(overlay=True)


## Exercises

- What changes if you don't enter a ``H_over_Z`` value in ``CleanRotator``?
- What happens if you don't set ``DataCleaner's`` ``max_freq``?  Is this good?
- What happens if you don't include ``inv`` when creating the SpectralDensity object?

